### Try-it 8.1: The "Best" Model

This module was all about regression and using Python's scikitlearn library to build regression models.  Below, a dataset related to real estate prices in California is given. While many of the assignments you have built and evaluated different models, it is important to spend some time interpreting the resulting "best" model.  


Your goal is to build a regression model to predict the price of a house in California.  After doing so, you are to *interpret* the model.  There are many strategies for doing so, including some built in methods from scikitlearn.  One example is `permutation_importance`.  Permutation feature importance is a strategy for inspecting a model and its features importance.  

Take a look at the user guide for `permutation_importance` [here](https://scikit-learn.org/stable/modules/permutation_importance.html).  Use  the `sklearn.inspection` modules implementation of `permutation_importance` to investigate the importance of different features to your regression models.  Share these results on the discussion board.

In [143]:
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
cali = pd.read_csv('data/housing.csv')

In [4]:
cali.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [5]:
cali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


First, let's see how much data there really is per column (NaaN etc.)

In [94]:
print(f'null values per column: {cali.isnull().sum()}')
print(f'NaaN values per column: {cali.isna().sum()}')

null values per column: longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64
NaaN values per column: longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64


Since 'total_bedrooms' is an important measure to use (a prediction), need to drop 207 rows where this value is null/Nan

In [100]:
# cali.shape # before: (20640, 10)
cali = cali.dropna(subset=['total_bedrooms'], axis='rows')
cali.shape # 20640 - 207 = 20433

(20433, 10)

'ocean_proximity' seems like an interesting feature (and is the only non-numeric column), lets see how many occurances there are of each 'type' to ensure we have one for every of the 20433 rows

In [102]:
# cali.groupby('ocean_proximity').count()
cali['ocean_proximity'].value_counts()
# <1H OCEAN     9034
# INLAND        6496
# NEAR OCEAN    2628
# NEAR BAY      2270
# ISLAND           5
# total: 20433

<1H OCEAN     9034
INLAND        6496
NEAR OCEAN    2628
NEAR BAY      2270
ISLAND           5
Name: ocean_proximity, dtype: int64

Since 'ocean_proximity' is non-numerical, we need to encode it: NOTE - I tried using ColumnTransformer but was not able to 'extract'/'set' the column names based on the variables

In [106]:
cali = pd.get_dummies(cali, columns=['ocean_proximity'], drop_first=False)
cali

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,0,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,0,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,0,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,0,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,0,1,0,0,0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,0,1,0,0,0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,0,1,0,0,0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,0,1,0,0,0


Now, we will split the data into the 'train' & 'test' sets

In [117]:
X = cali.drop('median_house_value', axis=1)
y = cali['median_house_value']

# split the data into data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# check the data set shapes
print(X_train.shape) # (16346, 13)
print(X_test.shape)  # (4087, 13)
print(y_train.shape) # (16346,)
print(y_test.shape)  # (4087,)
print(type(X_train), type(y_train)) # DataFrame, Series

(16346, 13)
(4087, 13)
(16346,)
(4087,)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


Lets establish a rough baseline using mean()

In [108]:
baseline_train = np.ones(shape=y_train.shape) * y_train.mean()
baseline_test = np.ones(shape=y_test.shape) * y_test.mean()

mse_baseline_train = mean_squared_error(baseline_train, y_train) # 13365445022.548538
mse_baseline_test = mean_squared_error(baseline_test, y_test)   #  13229470038.45748

print(f'mse_baseline_train {mse_baseline_train} mse_baseline_test {mse_baseline_test}')

mse_baseline_train 13365445022.548538 mse_baseline_test 13229470038.45748


Lets use linear regression & mean_squared_error on the original data.

In [118]:
lr_model = LinearRegression().fit(X_train, y_train)
y_predictions = lr_model.predict(X_test)
mse_original = mean_squared_error(y_test, y_predictions)
mse_original # 4594891091.607456

# mse_baseline_train: 13365445022.548538
# mse_baseline_test:  13229470038.45748
# mse_original:        4594891091.607456

4521118397.970796

This is a very high value. I am not sure but perhaps we need to drop the 'ocean_proximity'

In [131]:
cali = pd.read_csv('data/housing.csv')
cali = cali.drop(columns=['ocean_proximity'])
cali = cali.dropna(subset=['total_bedrooms'], axis='rows')
# cali.shape # (20433, 9)

X = cali.drop('median_house_value', axis=1)
y = cali['median_house_value']

# split the data again
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# check the data set shapes
print(X_train.shape) # (16346, 13)
print(X_test.shape)  # (4087, 13)
print(y_train.shape) # (16346,)
print(y_test.shape)  # (4087,)
print(type(X_train), type(y_train)) # DataFrame, Series

# establish the baseline using mean()
baseline_train = np.ones(shape=y_train.shape) * y_train.mean()
baseline_test = np.ones(shape=y_test.shape) * y_test.mean()

mse_baseline_train = mean_squared_error(baseline_train, y_train) # 13449835694.069315
mse_baseline_test = mean_squared_error(baseline_test, y_test)   #  12821839131.688322

print(f'mse_baseline_train {mse_baseline_train} mse_baseline_test {mse_baseline_test}')

# get the first MSE
lr_model = LinearRegression().fit(X_train, y_train)
y_predictions = lr_model.predict(X_test)
mse_original = mean_squared_error(y_test, y_predictions)
mse_original # 4620040373.408079

(16346, 8)
(4087, 8)
(16346,)
(4087,)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
mse_baseline_train 13449835694.069315 mse_baseline_test 12821839131.688322


4620040373.408079

Although I thought that introducing the 'dummies' for 'ocean_proximity' was creating a very high MSE, it appears that was not the culpret.

In [132]:
# what are the top 4 numeric correlations w/ 'median_house_value'?
four_highest_correlations = cali.corr(numeric_only=True)[['median_house_value']].nlargest(columns='median_house_value', n=5)
four_highest_correlations

,median_house_value
median_house_value,1.000000
median_income,0.688355
total_rooms,0.133294
housing_median_age,0.106432
households,0.064894


Determine multicolliniarity in the data using VIF; which column(s) to drop.

In [135]:
# from office hours 8/3/23
# def sk_vif(ind_variables, data):
#     vif_dict = {}
#     for ind_var in ind_variables:
#         # split the dataset: one variable against all others
#         not_ind_var = [i for i in ind_variables if i != ind_var]
#         X, y = data[not_ind_var], data[ind_var]
#         # fit model and compute R^2
#         r_squared = LinearRegression().fit(X, y).score(X, y)
#         # compite VIF
#         vif = 1/(1-r_squared)
#         vif_dict[ind_var] = vif
#     return pd.DataFrame({'VIF':vif_dict})

# print('VIF for original dataset')
# sk_vif(X.columns, X).sort_values(by='VIF', ascending=False)
def sk_vif(exogs, data):
    vif_dict = {}

    for exog in exogs:
        not_exog = [i for i in exogs if i !=exog]
        # split the dataset, one independent variable against all others
        X, y = data[not_exog], data[exog]

        # fit the model and obtain R^2
        r_squared = LinearRegression().fit(X,y).score(X,y)

        # compute the VIF
        vif = 1/(1-r_squared)
        vif_dict[exog] = vif

    return pd.DataFrame({"VIF": vif_dict})

In [136]:
sk_vif(X.columns, X).sort_values(by='VIF', ascending=False)

,VIF
total_bedrooms,36.003726
households,35.136045
total_rooms,12.717000
latitude,8.828919
longitude,8.713740
population,6.371238
median_income,1.731511
housing_median_age,1.260015


In [ ]:
We want to drop columns that have a VIF value larger than 5; doing so one at a time & re-running the sk_vif method

In [137]:
# drop 'total_bedrooms' & re-run sk_vif
X1 = X.drop(columns='total_bedrooms')
sk_vif(X1.columns, X1).sort_values(by='VIF', ascending=False)

,VIF
households,11.673860
total_rooms,9.583814
latitude,8.822928
longitude,8.677840
population,5.925295
median_income,1.524467
housing_median_age,1.257568


In [138]:
# drop 'households & re-run sk_vif'
X2 = X1.drop(columns='households')
sk_vif(X2.columns, X2).sort_values(by='VIF', ascending=False)

,VIF
latitude,8.316903
longitude,8.197084
total_rooms,4.725402
population,4.479289
median_income,1.308668
housing_median_age,1.257530


In [140]:
# drop 'latitude' & re-run sk_vif
X3 = X2.drop(columns='latitude')
sk_vif(X3.columns, X3).sort_values(by='VIF', ascending=False)
X3

,longitude,housing_median_age,total_rooms,population,median_income
0,-122.23,41.0,880.0,322.0,8.3252
1,-122.22,21.0,7099.0,2401.0,8.3014
2,-122.24,52.0,1467.0,496.0,7.2574
3,-122.25,52.0,1274.0,558.0,5.6431
4,-122.25,52.0,1627.0,565.0,3.8462
...,...,...,...,...,...
20635,-121.09,25.0,1665.0,845.0,1.5603
20636,-121.21,18.0,697.0,356.0,2.5568
20637,-121.22,17.0,2254.0,1007.0,1.7000
20638,-121.32,18.0,1860.0,741.0,1.8672


Finally! We no longer have VIF values above 5

We have to start over, dropping these columns before we can spilt the 'remaining' data set




In [152]:
cali = pd.read_csv('data/housing.csv')
cali = cali.drop(columns=['ocean_proximity'])
cali = cali.dropna(subset=['total_bedrooms'], axis='rows')

# drop 'total_bedrooms' (VIF)
cali = cali.drop(columns='total_bedrooms')
# drop 'households' (VIF)
cali = cali.drop(columns='households')
# drop 'latitude' (VIF)
cali = cali.drop(columns='latitude')

# before we split the data, let's see what are the correlations with this reduced data set in relation to 'median_house_value'
four_highest_correlations = cali.corr(numeric_only=True)[['median_house_value']].nlargest(columns='median_house_value', n=5)
# median_income	        0.688355
# total_rooms	        0.133294
# housing_median_age	0.106432
# population	       -0.025300

# re-create the data sets from X3
X = cali.drop('median_house_value', axis=1)
y = cali['median_house_value']

# split the data into data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# run linear regression
lr_model = LinearRegression().fit(X_train, y_train)
train_mse = mean_squared_error(y_train, lr_model.predict(X_train))
test_mse = mean_squared_error(y_test, lr_model.predict(X_test))

print(f'train_mse: {train_mse} test_mse: {test_mse}')

train_mse: 6506361440.671848 test_mse: 6136419867.839057


Both the mean_absolute_error & mean_squared_error remain very high, lets try changin the train_test_split parameters

In [153]:
# re-create the data sets from X3
X = cali.drop('median_house_value', axis=1)
y = cali['median_house_value']

# split the data into data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=22)

lr_model = LinearRegression().fit(X_train, y_train)
train_mse = mean_squared_error(y_train, lr_model.predict(X_train))
test_mse = mean_squared_error(y_test, lr_model.predict(X_test))

print(f'train_mse: {train_mse} test_mse: {test_mse}')

train_mse: 6463777694.230981 test_mse: 6358749210.176641
